## Data parsing for Tableau Project (DataViz)

In [2]:
library(tidyr)
library(ggplot2)
library(dplyr)

In [3]:
## Since the data are in Portuguese, we need to define the appropriate 'fileEncoding'
smp_total_2005 <- read.csv('Acessos_SMP_2005-200901_-_Total.csv', fileEncoding = 'ISO-8859-1', sep = ';')
smp_total_2009 <- read.csv('Acessos_SMP_200902-2014_-_Total.csv', fileEncoding = 'ISO-8859-1', sep = ';')
smp_total_2015 <- read.csv('Acessos_SMP_2015-2018_-_Total.csv', fileEncoding = 'ISO-8859-1', sep = ';')

In [35]:
nrow(smp_total_2015['2017.01'])

[1] 7743

In [4]:
head(smp_total_2015,3)

Empresa,CNPJ,Grupo,Região,DDD,UF,Tecnologia,Tipo,X2015.01,X2015.02,⋯,X2017.08,X2017.09,X2017.10,X2017.11,X2017.12,X2018.01,X2018.02,X2018.03,X2018.04,X2018.05
ALGAR CELULAR S/A,5.835916e+12,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,AMPS,Banda Estreita Móvel,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
ALGAR CELULAR S/A,5.835916e+12,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,CDMA,Banda Estreita Móvel,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ALGAR CELULAR S/A,5.835916e+12,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,CDMA2000,Banda Larga Móvel,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


### Cleaning the column names that refer to date:

In [5]:
head(names(smp_total_2005), 10)

[1] "Razão.Social" "Grupo"        "Região"       "Tecnologia"   "Tipo"        
 [6] "X2005.02"     "X2005.03"     "X2005.04"     "X2005.05"     "X2005.06"

In [6]:
head(names(smp_total_2009), 10)

[1] "Empresa"    "Grupo"      "Região"     "DDD"        "UF"        
 [6] "Tecnologia" "Tipo"       "X2009.02"   "X2009.03"   "X2009.04"

In [7]:
head(names(smp_total_2015), 10)

[1] "Empresa"    "CNPJ"       "Grupo"      "Região"     "DDD"       
 [6] "UF"         "Tecnologia" "Tipo"       "X2015.01"   "X2015.02"

Since there is no UF information in the first dataset, we will discard it. Also, in the second dataset there is "CNPJ" information, which is not relevant for this analysis and so it will also be removed. 

In [8]:
smp_total_2015$CNPJ <- NULL

We need now to remove the 'X' in front of the column names as well as converting them to Date format:  
This [reference](https://uc-r.github.io/tidyr) was really helpful on teaching me how to use the gather() method in the right way.

In [9]:
## For the first dataset:
colnames(smp_total_2009) <- gsub('X', '', colnames(smp_total_2009), fixed=TRUE)
## For the second dataset: 
colnames(smp_total_2015) <- gsub('X', '', colnames(smp_total_2015), fixed=TRUE)

In [11]:
long_2009 <- smp_total_2009 %>% gather(Date, Count, '2009.02':'2014.12')
long_2015 <- smp_total_2015 %>% gather(Date, Count, '2015.01':'2018.05')

In [21]:
tail(long_2009,5)

,Empresa,Grupo,Região,DDD,UF,Tecnologia,Tipo,Date,Count
398164,VIVO S.A.,TELEFÔNICA,Sul,55,RS,Dados,Banda Estreita Móvel,2014.12,NA
398165,VIVO S.A.,TELEFÔNICA,Sul,55,RS,Dados > 256kbps,Banda Larga Móvel,2014.12,NA
398166,VIVO S.A.,TELEFÔNICA,Sul,55,RS,Dados até 256kbps,Banda Estreita Móvel,2014.12,NA
398167,VIVO S.A.,TELEFÔNICA,Sul,55,RS,M2M,Banda Estreita Móvel,2014.12,NA
398168,VIVO S.A.,TELEFÔNICA,Sul,55,RS,TDMA,Banda Estreita Móvel,2014.12,NA


Now, the next step is to clean the data dropping the NA values:

In [13]:
smp2009 <- drop_na(long_2009)
smp2015 <- drop_na(long_2015)

In [16]:
head(smp2015)

,Empresa,Grupo,Região,DDD,UF,Tecnologia,Tipo,Date,Count
2,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,CDMA,Banda Estreita Móvel,2015.01,0
6,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,Dados Banda Larga,Banda Larga Móvel,2015.01,1073
8,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,GSM,Banda Estreita Móvel,2015.01,7810
9,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,LTE,Banda Larga Móvel,2015.01,0
11,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,M2M Especial,Banda Estreita Móvel,2015.01,0
12,ALGAR CELULAR S/A,ALGAR (CTBC TELECOM),Centro-Oeste,64,GO,M2M Padrão,Banda Estreita Móvel,2015.01,635


To adjust the date format, we will add the first day of the month to each register:

In [17]:
smp2009$Date <- as.Date(paste(smp2009$Date,'.01', sep=''), format = "%Y.%m.%d")
smp2015$Date <- as.Date(paste(smp2015$Date,'.01', sep=''), format = "%Y.%m.%d")

In [18]:
tail(smp2015)

,Empresa,Grupo,Região,DDD,UF,Tecnologia,Tipo,Date,Count
315317,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,Dados Banda Larga,Banda Larga Móvel,2018-05-01,277
315318,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,GSM,Banda Estreita Móvel,2018-05-01,35001
315319,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,LTE,Banda Larga Móvel,2018-05-01,57010
315320,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,M2M Especial,Banda Estreita Móvel,2018-05-01,7182
315321,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,M2M Padrão,Banda Estreita Móvel,2018-05-01,7698
315322,TIM CELULAR S.A.,TELECOM ITALIA,Sul,55,RS,WCDMA,Banda Larga Móvel,2018-05-01,33883


### Exporting the data

In [24]:
write.csv(smp2009, file = 'smp_2009_tidy_v2.csv', row.names = FALSE)
write.csv(smp2015, file = 'smp_2015_tidy_v2.csv', row.names = FALSE)